In [4]:
!pip install langchain --quiet
!pip install openai --quiet
!pip install pdf2image --quiet
!pip install pdfminer.six --quiet
!pip install singlestoredb --quiet
!pip install tiktoken --quiet
!pip install unstructured --quiet

In [6]:
from langchain.document_loaders import OnlinePDFLoader

loader = OnlinePDFLoader("http://leavcom.com/pdf/DBpdf.pdf")

data = loader.load()

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

print (f"You have {len(data)} document(s) in your data")
print (f"There are {len(data[0].page_content)} characters in your document")

You have 1 document(s) in your data
There are 13040 characters in your document


In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 0)
texts = text_splitter.split_documents(data)

print (f"You have {len(texts)} pages")

You have 8 pages


In [9]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


In [10]:
from langchain.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings()

In [11]:
%%sql

CREATE DATABASE IF NOT EXISTS pdf_db;
USE pdf_db;
DROP TABLE IF EXISTS pdf_docs;

In [12]:
from langchain.vectorstores import SingleStoreDB

s2_password = "<password>"
s2_host = "<host>"

os.environ["SINGLESTOREDB_URL"] = f"admin:{s2_password}@{s2_host}:3306/pdf_db"

docsearch = SingleStoreDB.from_documents(
    texts,
    embedding,
    table_name = "pdf_docs",
)

In [13]:
query_text = "Will object-oriented databases be commercially successful?"

docs = docsearch.similarity_search(query_text)

print(docs[0].page_content)

I N D U S T R Y T R E N D S

Whatever Happened to Object-Oriented Databases?

are gaining in popularity and are ex- pected to outsell even relational data- bases by 2003. And OO databases (see the “OO Database Orientation” sidebar) are still minor players with solid but strictly niche markets. Sales of relational databases have grown considerably faster than the sales of OO databases, and annual worldwide RDBMS revenues are now about 50 times larger.

Rick Cattell, distinguished engineer at Sun Microsystems, indicated, “Object- oriented databases are doing just ﬁne, and the news of their demise is highly exag- gerated. While their market [share] isn’t as big, they continue to be used in areas like CAD (computer-aided design) and telecommunications, where RDBMSs are not well suited.”

Neal Leavitt

A couple of years ago, industry

However, said Michael Stonebraker, chief technology ofﬁcer at Informix and an ORDBMS proponent and pioneer, “ODBMSs occupy a small niche market that has no br

In [14]:
import openai

prompt = f"The user asked: {query_text}. The most similar text from the document is: {docs[0].page_content}"

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response['choices'][0]['message']['content'])

The document does not provide a clear answer to the question of whether object-oriented databases will be commercially successful. It presents differing opinions on the state of object-oriented databases and their market appeal. While some believe that object-oriented databases have a niche market and are not as widely used as relational databases, others argue that they continue to be used in specific industries such as CAD and telecommunications. There is also a mention of object-relational database management systems (ORDBMSs) potentially cornering the market within five years. Overall, the document suggests that the success of object-oriented databases in the commercial marketplace is not yet clear.
